<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [5]:
from stock_stream.utils.variables import var

Allowed Symbols: ['DIS', 'BA', 'LUV', 'SNOW', 'AAPL', 'C', 'KO', 'MCD', 'AMD', 'TSLA', 'NKE']


In [36]:
import pandas as pd
import json

def get_active_alerts():
    df = pd.read_sql(f"""select * from {var.db}.alerts where is_active=1;""",con=var.engine)
    need_tables=list(set([v for t in alerts.rule_json.apply(lambda x: [t[0] for tb in json.loads(x).values() for k,t in tb.items() if k in ("table1", "table2")]).tolist() 
     for v in t if v !="None"]))
    
    return df,need_tables

# def send_alert():
alerts,table_names = get_active_alerts()

# # table_names = pd.read_sql(f"""SELECT distinct TABLE_NAME from 
# #                     information_schema.columns where TABLE_SCHEMA='{var.db}'""", 
# #                     con=var.engine).TABLE_NAME.tolist()
# table_names = [t for t in table_names if "1m" in t or "2m" in t or "3m" in t or "5m" in t]
table_data = {table:pd.read_sql(f"""select * from {var.db}.{table};""", 
            con=var.engine,parse_dates=["stamp"]) for table in table_names}

        # break
                


# send_alert()

In [37]:
table_data["bars_3m"]["open"].tolist()[-1],table_data["ema_1m"]["EMA_20"].tolist()[-1]

(873.28, 868.4072671248709)

In [38]:
table_data["rsi_1m"]["RSI_14"].tolist()[-1]

64.28334968028646

In [40]:
send_to="viper.incarnation@gmail.com"


In [54]:
mails={}
for _,row in alerts.iterrows():
    aname = row.alert_name
    amsg = row.message
    
    rules = json.loads(row.rule_json)
    print(rules)
    rtxts = {}
    rtxt=""
    for r, rule in rules.items():    
        value_check=False
        for table,detail in rule.items():
            if table in ["table1", "table2"]:
                table=detail[0]
                detail=detail[1:]
            print(table, detail)                
            if table=="None":
                value_check=True
            elif table in table_names:
                if len(detail)==2:
                    column,row=detail
                    operator=""
                else:
                    column,row,operator = detail
                row = -int(row.split("-")[-1])
                
                curr_df = table_data[table]
                symbols = curr_df.symbol.unique()
                v1 = {symbol:curr_df.query(f"symbol=='{symbol}'")[column].tolist()[row] for symbol in symbols}
                
                
                if len(rtxts)==0:
                    rtxts={s:f"{v}{operator}" for s,v in v1.items()}
                else:   
                    rtxts = {s:v+f"{v1[s]}{operator}" for s,v in rtxts.items()}  
                    
                v1=curr_df[column].tolist()[row]
                rtxt+=f"{v1}{operator}"

            if value_check and table=="value":
                rtxt+=f"{detail}"
                rtxts = {s:v+f"{detail}" for s,v in rtxts.items()}
            if table=="cond" and detail is not None:
                rtxt+=f" {detail.lower()} "
                rtxt=rtxt.replace("not", "!=")
                rtxts = {s:v+f" {detail.lower()} " for s,v in rtxts.items()}
                rtxts = {s:v.replace("not","!=") for s,v in rtxts.items()}
            # print(rtxt)

    print(rtxt)
    print(rtxts)
    try:
        alert_result=eval(rtxt)
        if mails.get(aname) is None:
            mails[aname]=""
        for symb,rt in rtxts.items():
            try:
                res = eval(rtxt)
                if res:
                    mails[aname]+=f"""Alert {aname} matched for {symb}.\n {amsg} \n"""
            except:
                pass
            
            

        print(eval(rtxt))
        
    except:
        pass
        


{'rule_0': {'table1': ['bars_3m', 'open', 'N-1', '>'], 'table2': ['bars_3m', 'high', 'N-1'], 'value': 0.0, 'cond': None}}
bars_3m ['open', 'N-1', '>']
bars_3m ['high', 'N-1']
value 0.0
cond None
873.28>875.79
{'AAPL': '156.81>157.19', 'AMD': '93.87>94.27', 'BA': '150.47>151.0', 'C': '51.01>51.35', 'DIS': '112.5>112.5', 'KO': '64.51>64.51', 'LUV': '47.0>47.0', 'MCD': '248.92>248.92', 'NKE': '118.63>119.0', 'SNOW': '166.6>166.6', 'TSLA': '873.28>875.79'}
False
{'rule_0': {'table1': ['bars_3m', 'open', 'N-1', '>'], 'table2': ['ema_1m', 'EMA_20', 'N-1'], 'value': 0.0, 'cond': None}}
bars_3m ['open', 'N-1', '>']
ema_1m ['EMA_20', 'N-1']
value 0.0
cond None
873.28>868.4072671248709
{'AAPL': '156.81>156.15340767979086', 'AMD': '93.87>93.44365776903632', 'BA': '150.47>150.09816935293765', 'C': '51.01>51.138641247011535', 'DIS': '112.5>112.21569787604807', 'KO': '64.51>64.35443629737703', 'LUV': '47.0>46.913761708049236', 'MCD': '248.92>248.23789109847792', 'NKE': '118.63>118.0124180245464', 'S

In [60]:
# subject = ""
# body = ""
# for aname,content in mails.items():
#     if content!='':
# #         print(aname, content)
#         subject=aname
#         body=content
#         var.mailer.send(send_to,subject,body)
        

In [61]:
mails

{'Test1': '',
 'Test2': 'Alert Test2 matched for AAPL.\n Market is awesome. \nAlert Test2 matched for AMD.\n Market is awesome. \nAlert Test2 matched for BA.\n Market is awesome. \nAlert Test2 matched for C.\n Market is awesome. \nAlert Test2 matched for DIS.\n Market is awesome. \nAlert Test2 matched for KO.\n Market is awesome. \nAlert Test2 matched for LUV.\n Market is awesome. \nAlert Test2 matched for MCD.\n Market is awesome. \nAlert Test2 matched for NKE.\n Market is awesome. \nAlert Test2 matched for SNOW.\n Market is awesome. \nAlert Test2 matched for TSLA.\n Market is awesome. \n',
 'Test3': 'Alert Test3 matched for AAPL.\n  \nAlert Test3 matched for AMD.\n  \nAlert Test3 matched for BA.\n  \nAlert Test3 matched for C.\n  \nAlert Test3 matched for DIS.\n  \nAlert Test3 matched for KO.\n  \nAlert Test3 matched for LUV.\n  \nAlert Test3 matched for MCD.\n  \nAlert Test3 matched for NKE.\n  \nAlert Test3 matched for SNOW.\n  \nAlert Test3 matched for TSLA.\n  \n',
 'Test4': 'Ale

In [26]:
[t[0] for tb in json.loads(alerts.rule_json[0]).values() for k,t in tb.items() if k in ("table1", "table2")]

['bars_3m', 'bars_3m']

In [35]:
list(set([v for t in alerts.rule_json.apply(lambda x: [t[0] for tb in json.loads(x).values() for k,t in tb.items() if k in ("table1", "table2")]).tolist() 
 for v in t if v !="None"]))

['bars_3m', 'ema_1m', 'rsi_1m', 'bars_2m']